<a href="https://colab.research.google.com/github/AnnaKrukowski/WBS_Case-Study_Eniac_Discounts/blob/main/20250120_Eniac_data_cleaning_with_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning with Pandas

- Few basic data cleaning steps
- Focus: `orders` and `orderlines` and `products`datasets


In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None) # to increase the width of the columns

In [4]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

First creating a copy of the `orders` and `orderlines` DataFrames. This way any changes won't affect the original DataFrames.

In [5]:
orders_df = orders.copy()

In [6]:
orderlines_df = orderlines.copy()

One of the best ways to begin data cleaning is by exploring using `.info()`. This will show:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

## 1.&nbsp; Duplicates
Checking for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

Deleting these rows using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html).

In [6]:
# orders
orders_df.duplicated().sum()

0

In [7]:
# orderlines
orderlines_df.duplicated().sum()

0

No duplicate rows in either DataFrame, so no need to delete any rows.

# 2.&nbsp; `.info()`

In [8]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [9]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [10]:
print(f"5 missing values represents {((orders_df.total_paid.isna().sum() / orders_df.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


Other quick option: `.value_count(normalize=True)`

In [11]:
orders_df.total_paid.isna().value_counts(normalize=True)

,proportion
total_paid,
False,0.999978
True,0.000022


As there is such a tiny amount of missing values, these rows will be simply deleted, as there is enough data without them.

In [12]:
orders_df = orders_df.loc[~orders.total_paid.isna(), :]

If the number of missing values is considerable, the following options are available:
+ impute values
+ transfer values from other DataFrames if they are available there
+ delete values
+ or any number of other creative solutions

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [13]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [14]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

#### 4.1.2.&nbsp;`unit_price`

In [15]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

ValueError: Unable to parse string "1.137.99" at position 6

Converting `unit_price` to a numeric data type leads to `ValueError`, which describes that Pandas does not understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points.

In [17]:
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts() # cheking how man prices contain 2 decimal points

,count
unit_price,
False,257814
True,36169


Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [18]:
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


<ipython-input-18-d7cb20f99e43>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)


This is a somewhat difficult decision, as 12.3% is a considerable amount of data... and it could even be that a larger part of the data could be lost in the end. For this project, the rows will be deleted as we only have 2 weeks for this project and I would like to show quick, accurate results.

Each line of “Orderlines” represents a product in an order. For example, if order number 175 contains 3 different products, then order 175 would have 3 lines in “orderlines”, one line for each product. If 2 of these products have “normal” prices (14.99, 15.85) and 1 has a price with 2 decimal places (1.137.99), the entire order must be removed and not just the affected line. If only the line with 2 decimal places were removed, a later analysis of products and prices could be misleading.

Therefore, the order numbers associated with the rows with 2 decimal places must be found and then all associated rows removed.

In [19]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]

orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [20]:
orderlines_df.shape[0]

216250

There are still 216250 rows in orderlines to work with. This should be more than enough for the evaluation.

Now that all of the 2 decimal point prices have been removed, it's time to try again converting the column `unit_price` to the correct datatype.

In [23]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   id_order          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  float64       
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 13.2+ MB


# Clean the `products` DataFrame


In [24]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

First creating a copy of the `products` DataFrames. This way any changes won't affect the original DataFrames.

In [25]:
products_df = products.copy()

In [26]:
products_df.head()

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pro (2006/07/08).,34.99,31.99,1,1364


In [27]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


These will be the next steps:
* Duplicates
* Missing values
* Datatypes


## Duplicates

In [28]:
products_df.duplicated().sum()

8746

First getting rid of the duplicates.

In [29]:
products_df = products_df.drop_duplicates()

In [31]:
products_df.duplicated().sum()

0

## `.info()`

In [32]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


### Missing values
`.info()` above shows that there are missing values in column `desc` and `price`

#### `desc`

In [33]:
products_df["desc"].isna().sum()

7

7 is a very small number to have missing --> closer look below

In [35]:
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard drive and NAS",NaN,107,814.659,0,1298
16128,APP1622-A,"Open - Apple Smart Keyboard Pro Keyboard Folio iPad 9.7 """,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,"Open - Kanex USB-C Gigabit Ethernet Adapter MacBook 12 """,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror and Comic Blond,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador T6 Intelligent Wireless (cable),NaN,199.99,1.441.174,0,11905404


There are two options:
* quickly and easily remove these rows.
* Or, alternatively, the products names here are quite descriptive, so I'm tempted to just copy them to the description column, so that there is a description if its needed later. --> this will be the way forward

In [36]:
products_df.loc[products_df['desc'].isna(), 'desc'] = products_df.loc[products_df['desc'].isna(), 'name']

In [37]:
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type


#### `price`

In [38]:
products_df.price.isna().sum()

46

In [39]:
print(f"The missing values in price are {(products_df.price.isna().value_counts(normalize=True)[1] * 100).round(2)}% of all rows in the DataFrame")

The missing values in price are 0.43% of all rows in the DataFrame


<ipython-input-39-149a8976a30c>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"The missing values in price are {(products_df.price.isna().value_counts(normalize=True)[1] * 100).round(2)}% of all rows in the DataFrame")


Way forward:
+ simply delete these rows to ensure that the numbers are trustworthy in the final DataFrame because

*Afterall, the price is very important when investigating discounts*

Option 1: `.loc`

In [35]:
products_df = products_df.loc[~products['price'].isna()]

Option 2: `.dropna()`

In [ ]:
# products_df = products_df.dropna(subset=['price'])

### Data types

Output of `.info()` shows that both `price` and `promo_price` have been stored as objects and not as a numerical datatypes.

Other problems: both columns have some prices with 3 decimal places and others with 2 decimal points - the latter will prevent me from converting the datatype to numerical, so first I have to investigate and solve these problems.

#### `price`

First, let's see how many values are affected by the 2-decimal-dot problems or 3 decimal places.

In [40]:
price_problems_number = products_df.loc[(products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

542

In [42]:
print(f"The column price has in total {price_problems_number} wrong values. This is {round(((price_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column price has in total 542 wrong values. This is 5.12% of the rows of the DataFrame


5.12% is a reasonable amount of our data. However, the price column will be important to understanding discounts, so I'd like it to be very trustworthy as I am basing business decisions on it. Therefore, I'll delete these rows

In [43]:
products_df = products_df.loc[(~products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :]

In [44]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pro (2006/07/08).,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horizontally for Apple Watch,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,0,1392


To complete our task, let's convert the column to a numeric datatype

In [46]:
products_df["price"] = pd.to_numeric(products_df["price"])

<ipython-input-46-f97981956831>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df["price"] = pd.to_numeric(products_df["price"])


In [47]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10038 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10038 non-null  object 
 1   name         10038 non-null  object 
 2   desc         10038 non-null  object 
 3   price        9992 non-null   float64
 4   promo_price  10038 non-null  object 
 5   in_stock     10038 non-null  int64  
 6   type         9992 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 627.4+ KB


#### `promo_price`

Again, let's begin by seeing how many values are affected by the 2-decimal-dots problem, or the 3 decimal-places problem

In [48]:
promo_problems_number = products_df.loc[(products_df.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9275

In [49]:
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9275 wrong values. This is 92.4% of the rows of the DataFrame


That's a lot of wrong data. Let's have a quick investigate to check that's correct. I'll make a DataFrame by copy-pasting the code I used above and then look at a large sample to check that all the numbers in the `promo_price` column really have either 2 decimal points or 3 decimal places.

In [50]:
promo_price_df = products_df.loc[(products_df.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :]
promo_price_df.sample(50)

,sku,name,desc,price,promo_price,in_stock,type
12142,MOP0075,Mophie Power Reserve micro USB External Battery 1350mAh Black,small compact external battery and portable device with USB port and microUSB,44.00,149.895,1,1515
13623,MUJ0018,Mujjo iPhone Leather Case Leather Champagne 8/7/6,ultrathin Case for iPhone vegetable tanned leather 7,39.90,359.902,1,11865403
1616,OLL0018,Olloclip 4-in-1 lens iPhone 6 / 6S and 6 / 6S Plus Silver / Black,Wide angle fisheye macro macro x10 and x15 for iPhone 6 / 6S and 6 / 6S Plus.,79.99,639.896,1,11905404
15953,AP20079,"Like new - Apple MacBook Pro Retina 13 ""i5 27 GHz | 8GB RAM | 128GB Flash",Computer Refurbished MacBook Pro 13-inch i5 27GHz | 8GB RAM | 128GB Flash,1499.00,11.555.936,0,"2,17E+11"
363,APP0434,Apple 5W USB power adapter iPhone and iPod,USB Power Adapter 5W 1A for iPhone iPod and Apple Watch.,25.00,240.004,1,14365395
18708,APP2506-A,Open - Silicon Case Cover Apple iPhone 8/7 Cobalt Blue,Ultrathin silicone case and microfiber premium for iPhone 8/7,39.00,314.664,0,11865403
7657,PAC1623,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 16GB RAM | 2TB SSD | Radeon R9 M395X 4GB",Desktop computer iMac 27-inch 3.3GHz Core i5 5K Retina | 16GB RAM | 2TB SSD | R9 M395X 4GB (MK482Y / A),4609.00,35.749.898,0,"5,74E+15"
18372,BEL0340,Belkin Cable HDMI High Speed ​​with Ethernet 5m,Belkin Cable High Speed ​​HDMI - Ethernet 5m for Mac,29.99,269.903,1,1325
11800,OWC0183,"SSD expansion kit OWC Aura Pro 6G 480GB Retina MacBook Pro / Macbook Air 11 ""and 13"" 2013-2015",Kit expansion SSD 480GB MacBook Retina 13 inches and 15 inches Late 2013 to 2015 and Mid Ipads 11 inches and 13 inches Mid 2013 to 2015 Early tools,568.99,4.945.839,0,12215397
11325,XDO0026,"X-Doria Dash Folio iPad Case Simple Pro 129 ""White",Leather case and polycarbonate sheet format and support for 129-inch iPad Pro.,34.99,249.901,0,12635403


So I was correct, over 90% of the data in this column is corrupt. There's no point deleting all of these rows, then I would barely have a products table. Instead, as it's only this column that appears to be very untrustworthy, I will delete the column.

In [51]:
products_cl = products_df.drop(columns=["promo_price"])

In [52]:
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10038 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       10038 non-null  object 
 1   name      10038 non-null  object 
 2   desc      10038 non-null  object 
 3   price     9992 non-null   float64
 4   in_stock  10038 non-null  int64  
 5   type      9992 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 549.0+ KB


Obviously, there's now no need to convert `promo_price` to a numerical datatype

Last step:
+ Saving the cleaned version of the DataFrames

In [53]:
from google.colab import files

orders_df.to_csv("orders_cl.csv", index=False)
files.download("orders_cl.csv")

orderlines_df.to_csv("orderlines_cl.csv", index=False)
files.download("orderlines_cl.csv")

products_cl.to_csv("products_cl.csv", index=False)
files.download("products_cl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>